In [1]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
RAW_DIR = PROJECT_ROOT / "data" / "raw"
ERA5_DIR = RAW_DIR / "era5_sat"
ERA5_DIR.mkdir(parents=True, exist_ok=True)

era5_t2m_path = ERA5_DIR / "era5_t2m_monthly_1979_2022_arctic.nc"
era5_sat_anom_path = ERA5_DIR / "era5_sat_anom_monthly_1979_2022_arctic.nc"

print("ERA5 t2m ->", era5_t2m_path)
print("SAT anomaly ->", era5_sat_anom_path)


ERA5 t2m -> /mnt/c/Users/Lenovo/Desktop/DS从入门到退役/LTL的DS生活/DS开源项目/physics-consistent-arctic-sea-ice-forecast/data/raw/era5_sat/era5_t2m_monthly_1979_2022_arctic.nc
SAT anomaly -> /mnt/c/Users/Lenovo/Desktop/DS从入门到退役/LTL的DS生活/DS开源项目/physics-consistent-arctic-sea-ice-forecast/data/raw/era5_sat/era5_sat_anom_monthly_1979_2022_arctic.nc


In [2]:
import cdsapi

if era5_t2m_path.exists():
    print("Already exists, skip download:", era5_t2m_path)
else:
    c = cdsapi.Client()

    request = {
        "product_type": "monthly_averaged_reanalysis",
        "variable": "2m_temperature",
        "year": [str(y) for y in range(1979, 2023)],
        "month": [f"{m:02d}" for m in range(1, 13)],
        "time": "00:00",
        # [North, West, South, East]
        "area": [90, -180, 60, 180],
        "format": "netcdf",
    }

    print("Requesting ERA5 monthly t2m (this can take a while)...")
    c.retrieve(
        "reanalysis-era5-single-levels-monthly-means",
        request,
        str(era5_t2m_path),
    )
    print("Downloaded:", era5_t2m_path)


2025-12-20 05:00:45,725 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


Requesting ERA5 monthly t2m (this can take a while)...


2025-12-20 05:00:47,300 INFO Request ID is ab0079a0-6412-4038-bd18-51bac7e9145d
2025-12-20 05:00:47,822 INFO status has been updated to accepted
2025-12-20 05:00:58,024 INFO status has been updated to running
2025-12-20 05:02:07,688 INFO status has been updated to successful


45f77b98e8613c5c0619093e96dda53b.nc:   0%|          | 0.00/109M [00:00<?, ?B/s]

Downloaded: /mnt/c/Users/Lenovo/Desktop/DS从入门到退役/LTL的DS生活/DS开源项目/physics-consistent-arctic-sea-ice-forecast/data/raw/era5_sat/era5_t2m_monthly_1979_2022_arctic.nc


In [ ]:
import xarray as xr
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
ERA5_DIR = PROJECT_ROOT / "data" / "raw" / 
"era5_sat"

t2m_path = ERA5_DIR / "era5_t2m_monthly_1979_2022_arctic.nc"
anom_path = ERA5_DIR / "era5_sat_anom_monthly_1979_2022_arctic.nc"

ds = xr.open_dataset(t2m_path)

# ⚠️ 不要 print(ds)
print("Dataset loaded.")
print("Sizes:", ds.sizes)
print("Variables:", list(ds.data_vars))

t2m = ds["t2m"] if "t2m" in ds else ds[list(ds.data_vars)[0]]
print("Using variable:", t2m.name)

clim = t2m.groupby("time.month").mean("time")
sat_anom = (t2m.groupby("time.month") - clim).astype("float32")
sat_anom = sat_anom.rename("sat_anom")

out = xr.Dataset({"sat_anom": sat_anom})
out.to_netcdf(anom_path)

print("Saved SAT anomaly to:", anom_path)
